In [1]:
import os
import argparse
import time

from pennylane import numpy as np
from sklearn.preprocessing import LabelEncoder
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from data_generator import gen_mel
from models import cnn_Model, dense_Model, attrnn_Model
from helper_q_tool import gen_qspeech, plot_acc_loss, show_speech

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"]="1"
data_ix = time.strftime("%m%d_%H%M")
train_audio_path = 'dataset/'
SAVE_PATH = "data_quantum/"

In [3]:
labels = [
    'left', 'go', 'yes', 'down', 'up', 'on', 'right', 'no', 'off', 'stop',
]

In [10]:
# parser.add_argument("--eps", type = int, default = 30, help = "Epochs")
# parser.add_argument("--bsize", type = int, default = 16, help = "Batch Size")
# parser.add_argument("--sr", type = int, default = 16000, help = "Sampling Rate for input Speech")
# parser.add_argument("--net", type = int, default = 1, help = "(0) Dense Model, (1) U-Net RNN Attention")
# parser.add_argument("--mel", type = int, default = 0, help = "(0) Load Demo Features, (1) Extra Mel Features")
# parser.add_argument("--quanv", type = int, default = 0, help = "(0) Load Demo Features, (1) Extra Mel Features")
# parser.add_argument("--port", type = int, default = 100, help = "(1/N) data ratio for encoding ")
epochs=30
batch_size=16
sampling_rate=16000
network='UNET'
mel='MEL'
quanv='MEL'
port=100

In [11]:
def gen_train(labels, train_audio_path, sr, port):
    all_wave, all_label = gen_mel(labels, train_audio_path, sr, port)

    label_enconder = LabelEncoder()
    y = label_enconder.fit_transform(all_label)
    classes = list(label_enconder.classes_)
    y = keras.utils.to_categorical(y, num_classes=len(labels))

    from sklearn.model_selection import train_test_split
    x_train, x_valid, y_train, y_valid = train_test_split(np.array(all_wave),np.array(y),stratify=y,test_size = 0.2,random_state=777,shuffle=True)
    h_feat, w_feat, _ = x_train[0].shape
    np.save(SAVE_PATH + "n_x_train_speech.npy", x_train)
    np.save(SAVE_PATH + "n_x_test_speech.npy", x_valid)
    np.save(SAVE_PATH + "n_y_train_speech.npy", y_train)
    np.save(SAVE_PATH + "n_y_test_speech.npy", y_valid)
    print("===== Shape", h_feat, w_feat)

In [12]:
def gen_quanv(x_train, x_valid, kr):
    print("Kernal = ", kr)
    q_train, q_valid = gen_qspeech(x_train, x_valid, kr)

    np.save(SAVE_PATH + "demo_t1.npy", q_train)
    np.save(SAVE_PATH + "demo_t2.npy", q_valid)

    return q_train, q_valid

In [13]:
if mel == 'MEL':
    x_train, x_valid, y_train, y_valid = gen_train(labels, train_audio_path, sampling_rate, port)
elif mel == 'DEMO':
    x_train = np.load(SAVE_PATH + "x_train_demo.npy")
    x_valid = np.load(SAVE_PATH + "x_test_demo.npy")
    y_train = np.load(SAVE_PATH + "y_train_demo.npy")
    y_valid = np.load(SAVE_PATH + "y_test_demo.npy")

  0%|          | 0/10 [00:01<?, ?it/s]


TypeError: melspectrogram() takes 0 positional arguments but 1 positional argument (and 4 keyword-only arguments) were given

In [8]:
if quanv == 'MEL':
    q_train, q_valid = gen_quanv(x_train, x_valid, 2)
elif mel == 'DEMO':
    q_train = np.load(SAVE_PATH + "q_train_demo.npy")
    q_valid = np.load(SAVE_PATH + "q_test_demo.npy")

Kernal =  2
Quantum pre-processing of train Speech:


KeyboardInterrupt: 

In [10]:
early_stop = EarlyStopping(monitor='val_loss', mode='min',
                           verbose=1, patience=10, min_delta=0.0001)

checkpoint = ModelCheckpoint('checkpoints/best_demo.hdf5', monitor='val_acc',
                             verbose=1, save_best_only=True, mode='max')

In [11]:
if network == 'DENSE':
    model = dense_Model(x_train[0], labels)
elif network == 'UNET':
    model = attrnn_Model(q_train[0], labels)

/Users/gabrielemorello/miniforge3/envs/DT2119-final-project/lib/python3.10/site-packages/keras/src/optimizers/legacy/gradient_descent.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [12]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 30, 63, 4)]          0         []                            
                                                                                                  
 batch_normalization (Batch  (None, 30, 63, 4)            16        ['input_2[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 permute (Permute)           (None, 63, 30, 4)            0         ['batch_normalization[0][0]'] 
                                                                                                  
 conv2d (Conv2D)             (None, 63, 30, 16)           336       ['permute[0][0]']         

In [13]:
history = model.fit(
    x=q_train,
    y=y_train,
    epochs=epochs,
    callbacks=[checkpoint],
    batch_size=batch_size,
    validation_data=(q_valid,y_valid)
)

Epoch 1/30


2023-05-16 16:31:39.976543: W tensorflow/core/grappler/costs/op_level_cost_estimator.cc:693] Error in PredictCost() for the op: op: "Softmax" attr { key: "T" value { type: DT_FLOAT } } inputs { dtype: DT_FLOAT shape { unknown_rank: true } } device { type: "CPU" model: "0" frequency: 2400 num_cores: 10 environment { key: "cpu_instruction_set" value: "ARM NEON" } environment { key: "eigen" value: "3.4.90" } l1_cache_size: 16384 l2_cache_size: 524288 l3_cache_size: 524288 memory_size: 268435456 } outputs { dtype: DT_FLOAT shape { unknown_rank: true } }


93/94 [============================>.] - ETA: 0s - loss: 2.1579 - accuracy: 0.1956

2023-05-16 16:31:45.633629: W tensorflow/core/grappler/costs/op_level_cost_estimator.cc:693] Error in PredictCost() for the op: op: "Softmax" attr { key: "T" value { type: DT_FLOAT } } inputs { dtype: DT_FLOAT shape { unknown_rank: true } } device { type: "CPU" model: "0" frequency: 2400 num_cores: 10 environment { key: "cpu_instruction_set" value: "ARM NEON" } environment { key: "eigen" value: "3.4.90" } l1_cache_size: 16384 l2_cache_size: 524288 l3_cache_size: 524288 memory_size: 268435456 } outputs { dtype: DT_FLOAT shape { unknown_rank: true } }


94/94 [==============================] - 9s 60ms/step - loss: 2.1557 - accuracy: 0.1947 - val_loss: 2.4969 - val_accuracy: 0.1100
Epoch 2/30
94/94 [==============================] - 5s 56ms/step - loss: 1.2208 - accuracy: 0.5800 - val_loss: 1.4702 - val_accuracy: 0.4920
Epoch 3/30
94/94 [==============================] - 5s 56ms/step - loss: 0.6357 - accuracy: 0.7840 - val_loss: 1.0988 - val_accuracy: 0.6500
Epoch 4/30
94/94 [==============================] - 5s 57ms/step - loss: 0.4663 - accuracy: 0.8353 - val_loss: 0.8860 - val_accuracy: 0.7240
Epoch 5/30
94/94 [==============================] - 5s 57ms/step - loss: 0.3867 - accuracy: 0.8747 - val_loss: 0.6841 - val_accuracy: 0.7720
Epoch 6/30
94/94 [==============================] - 6s 61ms/step - loss: 0.3140 - accuracy: 0.9013 - val_loss: 0.5910 - val_accuracy: 0.8280
Epoch 7/30
94/94 [==============================] - 5s 55ms/step - loss: 0.3135 - accuracy: 0.8953 - val_loss: 1.7589 - val_accuracy: 0.5800
Epoch 8/30
94/94 [======

In [14]:
model.save('checkpoints/'+ data_ix + '_demo.hdf5')

/Users/gabrielemorello/miniforge3/envs/DT2119-final-project/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [15]:
print("=== Batch Size: ", batch_size)

=== Batch Size:  16
